#DISCARDED DATASETS

This notebook's purpose is to compile the discarded datasets with have been deemed unsuitable for the project, and to print the datasets in order to find examples of wrongly classified tweets.

*Note: When saving or loading data from Drive, the paths are specific to my personal Drive*

In [ ]:
import pandas as pd
!pip install tweepy
import tweepy
import numpy
import warnings
#In order to ignore warnings when overwriting row values in panda dataframes
warnings.filterwarnings('ignore')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


####Tweepy credentials

In [ ]:
CONSUMER_KEY = #
CONSUMER_SECRET = #
OAUTH_TOKEN = #
OAUTH_TOKEN_SECRET = #

In [ ]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True)

####Imports & functions

In [ ]:
#Returns exception message (for Tweepy tweet extraction)
def getExceptionMessage(msg):
    words = msg.split(' ')
    errorMsg = ""
    for index, word in enumerate(words):
        if index not in [0,1,2]:
            errorMsg = errorMsg + ' ' + word
    errorMsg = errorMsg.rstrip("\'}]")
    errorMsg = errorMsg.lstrip(" \'")
    return errorMsg

#DISCARDED DATA

In [ ]:
#Large Scale Crowdsourcing and Characterization of Twitter Abusive Behavior
df_abusive_text = pd.read_csv('/content/hatespeech_id_label_founta.csv', encoding='utf8', header= None)
#HASOC
#2019
df_hasoc_2019_train = pd.read_csv('/content/english_dataset.tsv', encoding='utf8', sep='\t')
df_hasoc_2019_test = pd.read_csv('/content/hasoc2019_en_test-2919.tsv', encoding='utf8', sep='\t')
#2020
df_hasoc_2020_train = pd.read_csv('/content/hasoc_2020_en_train.csv', encoding='utf8')
df_hasoc_2020_test = pd.read_csv('/content/hasoc_2020_en_test.csv', encoding='utf8')
#2021
df_hasoc_2021_train = pd.read_csv('/content/en_Hasoc2021_train.csv', encoding='utf8')

For each dataset, only text classified as hate speech will be shown. The main reason for not using this data is because text annotated as hateful is, often enough to not be useful, incorrectly judged.

###[Large Scale Crowdsourcing and Characterization of Twitter Abusive Behavior](https://arxiv.org/pdf/1802.00393.pdf)

####Obtaining tweet text

In [ ]:
#Split the single column into 2
df_abusive_text[['TweetID', 'hate speech']] = df_abusive_text[0].str.split(',', 1, expand=True)
df_abusive_text = df_abusive_text[['TweetID', 'hate speech']]

In [ ]:
df_abusive_text['text'] = "0"

In [ ]:
for idx, row in df_abusive_text.iterrows():
  #This condition is for when execution is stopped and later on resumed
  if not any(c.isalpha() for c in str(df_abusive_text['text'][idx])):
    try:
      if api.get_status(df_abusive_text['TweetID'][idx]):
        tweetFetched = api.get_status(df_abusive_text['TweetID'][idx])
        df_abusive_text['text'][idx] = tweetFetched.text
    except tweepy.TweepError as e:
      # print (e.api_code)
      # print (getExceptionMessage(e.reason))
      df_abusive_text['text'][idx] = "TWITTER__ERROR" + getExceptionMessage(e.reason)
    #We save the dataframe every line to not lose any progress (in case this cell would stop after 6+ hours!)
    df_abusive_text.to_csv('/content/drive/MyDrive/TFG/data/abusive_text_data.csv', index=False)  

####Rest of preprocessing


In [ ]:
df_abusive_text = pd.read_csv('/content/drive/MyDrive/TFG/data/abusive_text_data.csv', encoding='utf8', engine="python")
#Because of how long running time would be to extract all tweets, only 44.45% of the tweets have been extracted, which is 
#a good enough sample to see if the data has been properly classified

In [ ]:
df_abusive_text

,TweetID,hate speech,text
0,849667487180259329,abusive,TWITTER__ERRORNo status found with that ID.
1,850490912954351616,abusive,Alex Brosas another idiot #ALDUBKSGoesToUS ht...
2,848791766853668864,abusive,"RT @ItIzBiz: as Nancy Reagan would say, 'just ..."
3,848306464892604416,abusive,TWITTER__ERRORNo status found with that ID.
4,850010509969465344,normal,RT @MailOnline: The Nazi death gas so horrific...
...,...,...,...
100002,848922176128376832,hateful,0
100003,849405078956109829,abusive,0
100004,847981251143254016,normal,0
100005,850180983253073920,abusive,0


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [ ]:
for idx, row in df_abusive_text.iterrows():
  if "TWITTER__ERROR" in str(row.text) or "0" in str(row.text) or row["hate speech"]!="hateful":
    df_abusive_text = df_abusive_text.drop(labels=idx, axis=0)

In [ ]:
df_abusive_text['hate speech'] = df_abusive_text['hate speech'].replace(['hateful'],"1")
df_abusive_text.rename(columns = {'task_2':'hate speech'}, inplace = True)
df_abusive_text = df_abusive_text.drop(['TweetID'], axis=1)

In [ ]:
df_abusive_text

,TweetID,hate speech,text
5,850433664890544128,1,I hate er chase because if the Bitch that work...
12,850100742035836929,1,RT @nyctophil3: Pineapples do not belong on pi...
21,847804507367100416,1,Niggas keep talking about women wearing weave ...
33,850090319165050880,1,@vappywave idiot that's not gonna work. you go...
67,848686686930382848,1,RT @ayevonnn: bruh i fucking hate people like ...
...,...,...,...
44055,849005550524387328,1,lrt is cute because they're opposite elements ...
44088,850157792925241346,1,Voted for the biggest fucking liar @POTUS ever...
44348,848578067043889153,1,"Supposed ""Anti-fascist"" favors murderous commu..."
44371,850378195224465411,1,Syrian ally Iran blasts U.S. missile strikes a...


###[HASOC](https://hasocfire.github.io/hasoc/2021/index.html)

In [ ]:
#We just need the text and "task 2"'s classification, which indicated if the text contains hate speech 
df_hasoc_2019_train = df_hasoc_2019_train[['text', 'task_2']]
df_hasoc_2019_train['task_2'] = df_hasoc_2019_train['task_2'].replace(['HATE'],"1")
df_hasoc_2019_train.rename(columns = {'task_2':'hate speech'}, inplace = True)

In [ ]:
for idx, row in df_hasoc_2019_train.iterrows():
  if "1" != str(row["hate speech"]):
    df_hasoc_2019_train = df_hasoc_2019_train.drop(labels=idx, axis=0)

In [ ]:
df_hasoc_2019_test = df_hasoc_2019_test[['text', 'task_2']]
df_hasoc_2019_test['task_2'] = df_hasoc_2019_test['task_2'].replace(['HATE'],"1")
df_hasoc_2019_test.rename(columns = {'task_2':'hate speech'}, inplace = True)

In [ ]:
for idx, row in df_hasoc_2019_test.iterrows():
  if "1" != str(row["hate speech"]):
    df_hasoc_2019_test = df_hasoc_2019_test.drop(labels=idx, axis=0)

In [ ]:
df_hasoc_2019 = pd.concat([df_hasoc_2019_test, df_hasoc_2019_train])

In [ ]:
df_hasoc_2019

,text,hate speech
9,Jesus Christ Christian News. Illuminati is now...,1
11,"'In Kerala, it's all halal'. Of course, sic...",1
16,@Explore_Islam told you Islam is not fair. No ...,1
42,"Remember, it is the majority of the English wh...",1
54,Omar was allowed to take the oath of office on...,1
...,...,...
5840,Full of fools.. ICC @ICCMediaComms @ICC @crick...,1
5842,"The US-Mexican border, where established moral...",1
5844,@ICC All Eng matches are on dry pitches &amp; ...,1
5845,It seems the Rain &amp; thunder god only like ...,1


In [ ]:
#We just need the text and "task 2"'s classification, which indicated if the text contains hate speech 
df_hasoc_2020_train = df_hasoc_2020_train[['text', 'task2']]
df_hasoc_2020_train['task2'] = df_hasoc_2020_train['task2'].replace(['HATE'],"1")
df_hasoc_2020_train.rename(columns = {'task2':'hate speech'}, inplace = True)

In [ ]:
for idx, row in df_hasoc_2020_train.iterrows():
  if "1" != str(row["hate speech"]):
    df_hasoc_2020_train = df_hasoc_2020_train.drop(labels=idx, axis=0)

In [ ]:
df_hasoc_2020_test = df_hasoc_2020_test[['text', 'task2']]
df_hasoc_2020_test['task2'] = df_hasoc_2020_test['task2'].replace(['HATE'],"1")
df_hasoc_2020_test.rename(columns = {'task2':'hate speech'}, inplace = True)

In [ ]:
for idx, row in df_hasoc_2020_test.iterrows():
  if "1" != str(row["hate speech"]):
    df_hasoc_2020_test = df_hasoc_2020_test.drop(labels=idx, axis=0)

In [ ]:
df_hasoc_2020 = pd.concat([df_hasoc_2020_test, df_hasoc_2020_train])

In [ ]:
df_hasoc_2020

,text,hate speech
38,RT @canzaynnot: ARE YALL SEEING THIS???? ARE Y...,1
50,RT @JackPosobiec: BREAKING: President Trump pl...,1
96,RT @jazzyBOOvega: half of y’all don’t deserve ...,1
128,@edbanger1 @TfLTPH Some idiot sat in an office...,1
150,@BernieSanders Bernie will never be the Presid...,1
...,...,...
3494,RT @DVATW: British Muslims.\n*More than half o...,1
3531,RT @JoeyNoCollusion: You had a coup against th...,1
3537,@GOPChairwoman @realDonaldTrump Cause they nev...,1
3622,RT @evewhite5500: Father of Colorado school sh...,1


In [ ]:
#We just need the text and "task 2"'s classification, which indicated if the text contains hate speech 
df_hasoc_2021_train = df_hasoc_2021_train[['text', 'task_2']]
df_hasoc_2021_train['task_2'] = df_hasoc_2021_train['task_2'].replace(['HATE'],"1")
df_hasoc_2021_train.rename(columns = {'task_2':'hate speech'}, inplace = True)

In [ ]:
for idx, row in df_hasoc_2021_train.iterrows():
  if "1" != str(row["hate speech"]):
    df_hasoc_2021_train = df_hasoc_2021_train.drop(labels=idx, axis=0)

In [ ]:
df_hasoc_2021_train

,text,hate speech
8,"@Chahal_Shekhar Sorry we won't, why can't your...",1
23,"@SubhashiniAli @PoojaB1972 Wah @narendramodi ,...",1
30,It's an international shame for India as our P...,1
35,Modi government tied with twitter and removed ...,1
40,India becoming the first country to see more t...,1
...,...,...
3833,#BJP your supporters are openly crying and you...,1
3835,All the elections have been finished. No other...,1
3836,"@PritishNandy You are bereft of intellect, emo...",1
3839,India has suffered a lot. That Chinese bastard...,1
